In [1]:
import fileIO
import splice_files
import validate_timestamps
import coordinate_utilities
import time_utilities

from numpy import linspace
from math import ceil

In [4]:
# enter name and directory of input file
input_filename1  = 'workingDir/loterie_fixed.gpx'
input_filename2  = 'workingDir/beep2.gpx'

# select data fields to import
data_flags1 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
data_flags2 = {
    'gpsflag':  True,
    'timeflag': False,
    'eleflag':  True,
    'hrflag':   False, 
    'cadflag':  False, 
    'pwrflag':  False 
}

In [5]:
trkptlist1, meta_data1 = fileIO.import_trackpoints(data_flags1, input_filename1,skip_metadata=True)
trkptlist2, meta_data2 = fileIO.import_trackpoints(data_flags2, input_filename2,skip_metadata=True)

15027 points imported from workingDir/loterie_fixed.gpx.
81 points imported from workingDir/beep2.gpx.


In [6]:
trkptlist2 = trkptlist2[64:]

In [7]:
coordinate_utilities.compute_total_distance(trkptlist2)

502.03788725127714

From looking at the GPS gap in the broken file, we got the timestamps ('20:43:00', '20:51:28'). Since the watch was paused for a few minutes, I choose the earlier timestamp to be later. ~500m should be around 3 min.

In [9]:
ts1 = {'year': 2023, 'month': 12, 'day': 21, 'hour': 20, 'minute': 48, 'second': 50}
ts2 = {'year': 2023, 'month': 12, 'day': 21, 'hour': 20, 'minute': 51, 'second': 28}
gap_timestamps = time_utilities.generate_timepoints(ts1,ts2)
# print(gap_timestamps)
print(len(gap_timestamps))

159


Now define how many points need to be added between each node with N based on the number of timestamps.

In [21]:
trkptlist2_dense = coordinate_utilities.add_interpolated_points(trkptlist2,data_flags2,N=155)

Original number of points: 17, new number of points: 157.
Original distance: 502.03788725127714, new distance: 502.0378872514009.


Interpolated points are successfully added when the distance is preserved but the length has increased.

Generate timestamps for the missing GPX coordinates

Before splicing the files, let's fix the timestamps.

In [22]:
trkptlist2_dense = time_utilities.add_non_dummy_times(trkptlist2_dense,gap_timestamps)
validate_timestamps.compare_trkpt(trkptlist2_dense)

Matching 159 timestamps to 157 GPS points.
Timestamp start: 2023-12-21T20:48:50
Timestamp end:   2023-12-21T20:51:27          

Fail: 0, Success: 156


In [23]:
# one last check:
for trkpt  in trkptlist2_dense:
    if trkpt.hr == None:
       trkpt.hr = 0 

Export the file:

In [24]:
data_flags3 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
output_filename = "outputDir/garbagio2.gpx"
fileIO.export_trackpoints(data_flags3, trkptlist2_dense, output_filename)

Exporting file...
File exported to outputDir/garbagio2.gpx
